主程式碼

In [1]:
def crawler(url):
  import requests
  from bs4 import BeautifulSoup
  headers = {'user-agent':"123"}
  res = requests.get(url,headers=headers)
  soup = BeautifulSoup(res.text,'html.parser')
  return soup


In [2]:
def pttfund_csv():
  #創造csv
  import csv
  
  with open('ptt_fund.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['主文/留言','作者','時間','標題','推/噓','內容'])

In [3]:
def timer_articles(url):
  #判別文章時間，抓取狀態
  from datetime import datetime
  global start,end

  soup = crawler(url)
  header = soup.find_all('span','article-meta-value')
  date=header[-1].text.replace('  '," ")
  
  time=datetime.strptime(date,"%a %b %d %H:%M:%S %Y")
  start=datetime.strptime("Fri Aug 20 23:59:59 2021","%a %b %d %H:%M:%S %Y")
  end=datetime.strptime("Tue Mar 1 00:00:00 2022","%a %b %d %H:%M:%S %Y")

  if time < end:
    if time > start:
      return True,True 
    else:
      return False,False #從新往舊文章爬，已全數爬取完畢
  else:
    return False,True #從新往舊文章爬，尚未開始

In [13]:
def crawler_articles(url):
  #各別取得主文/留言
  from datetime import datetime

  author,time,title,contents,date=get_maininfo(url)

  #留言
  soup = crawler(url)
  article = soup.find_all('div','push')
  try:
    lastauthor=article[0].find('span', 'push-userid').getText()
  except:
    #文章沒有任何回覆
    lastauthor=''

  if lastauthor == '':
    if '[新聞]' not in title and '訪談招募' not in title:
      article_csv("主文",author,time,title,'無',contents) #將主文寫入csv 
  else:
    article_csv("主文",author,time,title,'無',contents) #將主文寫入csv


  pushtype='無'
  messages=''
  retitle=''

  for i in article:
    author=i.find('span', 'push-userid').getText()# 目前的回覆者
    emotion=i.find('span', 'push-tag').getText().strip()
    if author == lastauthor:
      if emotion == "噓" or emotion == "推":
        #針對主題的新回覆
        pushtype=emotion
        author,time,messages=get_reinfo(i,date,url)
      else:
        #同一則留言的回覆
        messages+=i.find('span', 'f3 push-content').getText().replace(': ','')
    else:
      header = soup.find_all('span','article-meta-value')
      date=datetime.strptime(header[3].text,"%a %b %d %H:%M:%S %Y")
      if emotion == "噓" or emotion == "推":
        #確定上一個人留言完畢了
        if retitle =="":
          article_csv("回覆",lastauthor,time,title,pushtype,messages)
          messages=''
        else:
          article_csv("回覆",lastauthor,time,retitle,pushtype,messages)
          retitle=''
          messages=''
        author,time,messages=get_reinfo(i,date,url)
      else:
        #新的人回覆上一個人的回覆
        article_csv("回覆",lastauthor,time,title,'無',messages)
        retitle=messages
        datetext=str(date.strftime("%Y"))+"/"+i.find('span', 'push-ipdatetime').getText().strip().replace('\n','')+":00"
        messages = i.find('span', 'f3 push-content').getText().replace(': ','')

    lastauthor=author
  if lastauthor != '':
    if retitle =="":
      article_csv("回覆",lastauthor,time,title,pushtype,messages)#將回覆寫入csv
    else:
      article_csv("回覆",lastauthor,time,retitle,pushtype,messages)
      retitle=''
      messages=''

In [5]:
def get_maininfo(url):
  #取得主文資訊
  from datetime import datetime
  import pandas as pd
  soup = crawler(url)
  header = soup.find_all('span','article-meta-value')

  author = header[0].text # 作者
  title = header[2].text # 標題

  # 日期判斷
  basic = soup.find_all('span','f2')
  rewrite=False
  for i in range(len(basic)-1,1,-1):
    if "編輯" in basic[i].text:
      rewrite=True
      time=basic[i].text.replace('\n','').split(",") 
      date=datetime.strptime(time[1]," %m/%d/%Y %H:%M:%S")
      time=date.strftime("%Y-%m-%d %H:%M:%S")
      break
  if not rewrite:
    date=datetime.strptime(header[3].text,"%a %b %d %H:%M:%S %Y") 
    time=date.strftime("%Y-%m-%d %H:%M:%S")#換成和dcard一樣的格式，2022-05-01 07:26:25
  
  # 內容
  main_container = soup.find(id='main-container')
  content = main_container.text.split('--')[0].split('\n')[2:]
  contents = '\n'.join(content)
  
  return author,time,title,contents,date

In [6]:
def get_reinfo(i,date,url):
  #取得留言資訊
  import requests
  from bs4 import BeautifulSoup
  from datetime import datetime

  soup = crawler(url)
  article = soup.find_all('div','push')

  author=i.find('span', 'push-userid').getText().strip() # 回覆者
  datetext=str(date.strftime("%Y"))+"/"+i.find('span', 'push-ipdatetime').getText().replace('\n','').strip()+":00" # 取得日期文字
  date=datetime.strptime(datetext,"%Y/%m/%d %H:%M:%S")
  time=date.strftime("%Y-%m-%d %H:%M:%S")
  messages = i.find('span', 'f3 push-content').getText().replace(': ','')# 留言內容

  return author,time,messages

In [7]:
def article_csv(type,author,time,title,pushtype,messages):
  #寫入csv
  import csv
  with open('ptt_fund.csv', 'a+', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([type,author,time,title,pushtype,messages])

In [14]:
import requests
from bs4 import BeautifulSoup
import time

url = "https://www.ptt.cc/bbs/Fund/index1088.html"
headers = {'user-agent':"123"}
soup = crawler(url)
pttfund_csv() #創建csv檔/覆蓋掉原內容

while url!="":
  print("開始一頁",url)
  articles = soup.select('.title a') #取得頁面所有文章
  for i in range(len(articles)-1,0,-1):
    each_title=articles[i]
    article_url = "https://www.ptt.cc" + each_title['href'] #取得文章網址

    #排除[公告]、版主聯署的文章
    if "[公告]" not in each_title.text and "推薦人" not in each_title.text and "[連署板主]" not in each_title.text:
      
      #判斷文章是否在要的時間區間內
      timezone, status = timer_articles(article_url) 
      if timezone:
        article_data=crawler_articles(article_url) #抓取頁面內容並寫入csv
        time.sleep(5) #和dcard一樣5秒爬1次新的主文

  if status:
    paging = soup.select('.btn-group-paging a')
    last_url = "https://www.ptt.cc" + paging[1]['href']
    url = last_url
    time.sleep(1) #和dcard一樣停1秒
    soup = crawler(url)
  else:
    url=""

開始一頁 https://www.ptt.cc/bbs/Fund/index1088.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1087.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1086.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1085.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1084.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1083.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1082.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1081.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1080.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1079.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1078.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1077.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1076.html
開始一頁 https://www.ptt.cc/bbs/Fund/index1075.html
